In [6]:
import rasterio
import numpy as np

def calculate_ndvi(file_path, nir_band=4, red_band=1):
    with rasterio.open(file_path) as src:
        nir = src.read(nir_band).astype(float)
        red = src.read(red_band).astype(float)
        np.seterr(divide='ignore', invalid='ignore')
        ndvi = (nir - red) / (nir + red)
        return ndvi

def calculate_ndwi(file_path, nir_band=4, green_band=3):
    with rasterio.open(file_path) as src:
        nir = src.read(nir_band).astype(float)
        green = src.read(green_band).astype(float)
        np.seterr(divide='ignore', invalid='ignore')
        ndwi = (green - nir) / (green + nir)
        return ndwi
    
    
def calculate_ndmi(file_path, nir_band=8, swir_band=7):
    with rasterio.open(file_path) as src:
        nir = src.read(nir_band).astype(float)
        swir = src.read(swir_band).astype(float)
        np.seterr(divide='ignore', invalid='ignore')
        ndmi = (nir - swir) / (nir + swir)



def calculate_savi(file_path, nir_band=8, red_band=6, L=0.5):
    with rasterio.open(file_path) as src:
        nir = src.read(nir_band).astype(float)
        red = src.read(red_band).astype(float)
        savi = ((nir - red) / (nir + red + L)) * (1 + L)
        return savi


        return ndmi

def save_index_to_tiff(index, src, output_filename):
    with rasterio.open(
        output_filename,
        'w',
        driver='GTiff',
        height=index.shape[0],
        width=index.shape[1],
        count=1,
        dtype=index.dtype,
        crs=src.crs,
        transform=src.transform,
    ) as dst:
        dst.write(index, 1)

# Path to the .tif file
file_path = '20161014_182328_0c37_3B_AnalyticMS.tif'

# Calculate indices
ndvi = calculate_ndvi(file_path)
ndwi = calculate_ndwi(file_path)
# savi = calculate_savi(file_path)
# ndmi = calculate_ndmi(file_path)

# Save indices as .tif files
with rasterio.open(file_path) as src:
    save_index_to_tiff(ndvi, src, 'ndvi_output.tif')
    save_index_to_tiff(ndwi, src, 'ndwi_output.tif')
    # save_index_to_tiff(savi, src, 'savi_output.tif')
    # save_index_to_tiff(ndmi, src, 'ndmi_output.tif')


In [16]:
import rasterio
import numpy as np

def calculate_true_color(file_path, red_band=3, green_band=2, blue_band=1):
    with rasterio.open(file_path) as src:
        red = src.read(red_band).astype(float)
        green = src.read(green_band).astype(float)
        blue = src.read(blue_band).astype(float)

        red_normalized = (red / 10000) * 2.5
        green_normalized = (green / 10000) * 2.5
        blue_normalized = (blue / 10000) * 2.5

        true_color = np.dstack((red_normalized, green_normalized, blue_normalized))
        return true_color


def save_true_color_to_tiff(true_color, src, output_filename):
    with rasterio.open(
        output_filename,
        'w',
        driver='GTiff',
        height=true_color.shape[0],
        width=true_color.shape[1],
        count=3,
        dtype=true_color.dtype,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        for i in range(true_color.shape[2]):
            dst.write(true_color[:, :, i], i + 1)


In [17]:
file_path = '20161014_182328_0c37_3B_AnalyticMS.tif'
output_filename = 'true_color_output.tiff'  

with rasterio.open(file_path) as src:
    true_color = calculate_true_color(file_path)
    save_true_color_to_tiff(true_color, src, output_filename)
